In [1]:
import time
import pyaudio
import wave

import numpy as np

In [2]:
import wave
import os,sys
import ctypes
import contextlib
import numpy as np
from ctypes import util
from scipy.io import wavfile
from pydub import AudioSegment

lib_path = util.find_library("rnnoise")
if (not("/" in lib_path)):
    lib_path = (os.popen('ldconfig -p | grep '+lib_path).read().split('\n')[0].strip().split(" ")[-1] or ("/usr/local/lib/"+lib_path))

lib = ctypes.cdll.LoadLibrary(lib_path)
lib.rnnoise_process_frame.argtypes = [ctypes.c_void_p,ctypes.POINTER(ctypes.c_float),ctypes.POINTER(ctypes.c_float)]
lib.rnnoise_process_frame.restype = ctypes.c_float
lib.rnnoise_create.restype = ctypes.c_void_p
lib.rnnoise_destroy.argtypes = [ctypes.c_void_p]

# borrowed from here 
# https://github.com/Shb742/rnnoise_python
class RNNoise(object):
    def __init__(self):
        self.obj = lib.rnnoise_create(None)
    def process_frame(self,inbuf):
        outbuf = np.ndarray((480,), 'h', inbuf).astype(ctypes.c_float)
        outbuf_ptr = outbuf.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
        VodProb =  lib.rnnoise_process_frame(self.obj,outbuf_ptr,outbuf_ptr)
        return (VodProb,outbuf.astype(ctypes.c_short).tobytes())

    def destroy(self):
        lib.rnnoise_destroy(self.obj)
        
denoiser = RNNoise()

In [3]:
FORMAT = pyaudio.paInt16
CHANNELS = 1
# RATE = 44100
RATE = 48000
CHUNK = 960/2 # CHUNK = 1024
# RECORD_SECONDS = 0.1
WAVE_OUTPUT_FILENAME = "output.wav"

audio = pyaudio.PyAudio()
denoiser = RNNoise()

In [4]:
def callback(in_data, frame_count, time_info, status):
    denoised_frame = denoiser.process_frame(in_data)[1]
    return (denoised_frame, pyaudio.paContinue)

stream = audio.open(format=FORMAT,\
                    channels=CHANNELS,\
                    rate=RATE,\
                    input=True,\
                    output=True,\
                    stream_callback=callback,\
                    # input_device_index=1, \
                    frames_per_buffer=CHUNK)

stream.start_stream()

time.sleep(30)
    
stream.stop_stream()
stream.close()

In [ ]:
WAVE_OUTPUT_FILENAME = "output.wav"

waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
waveFile.setnchannels(CHANNELS)
waveFile.setsampwidth(audio.get_sample_size(FORMAT))
waveFile.setframerate(RATE)

def callback(in_data, frame_count, time_info, status):  
    waveFile.writeframes(in_data)
    return (in_data, pyaudio.paContinue)

stream = audio.open(format=FORMAT,\
                    channels=CHANNELS,\
                    rate=RATE,\
                    input=True,\
                    stream_callback=callback,\
                    input_device_index=7, \
                    frames_per_buffer=CHUNK)

stream.start_stream()

time.sleep(10)
    
stream.stop_stream()
stream.close()

In [8]:
info = audio.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
for i in range(0, numdevices):
        if (audio.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
            print "Input Device id ", i, " - ", audio.get_device_info_by_host_api_device_index(0, i).get('name')

Input Device id  5  -  pulse
Input Device id  6  -  default
